In [ ]:
!python --version  #파이썬 버전 확인

Python 3.10.12


In [ ]:
# 필요한 패키지 설치
!apt-get update
!apt-get install -y wget unzip

# 크롬 브라우저 설치
!wget -O /tmp/chrome-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.76/linux64/chrome-linux64.zip
!unzip -o /tmp/chrome-linux64.zip -d /usr/local/
!ln -sf /usr/local/chrome-linux64/chrome /usr/bin/google-chrome  # 심볼릭 링크 강제로 덮어쓰기

# 크롬드라이버 설치
!wget -O /tmp/chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/125.0.6422.76/linux64/chromedriver-linux64.zip
!unzip -o /tmp/chromedriver.zip -d /usr/local/bin/
!mv /usr/local/bin/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
!chmod +x /usr/local/bin/chromedriver
!pip install selenium


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Fetched 3,732 kB in 7s (538 kB/s)
Reading package lists... Done

In [ ]:
import selenium
print(selenium.__version__)

4.21.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# 다운로드 경로 설정
base_download_dir = "/content/drive/My Drive/DownloadFile"

# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")  # GUI 없는 환경에서 사용
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--remote-debugging-port=9222')

# 여기서 Chromium 브라우저의 경로를 명시적으로 설정합니다.
options.binary_location = '/usr/local/chrome-linux64/chrome'


# ChromeDriver 경로 설정
chrome_driver_path = '/usr/local/bin/chromedriver'  # ChromeDriver 경로

# ChromeDriver 서비스 설정
service = Service(chrome_driver_path)

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 장학 공고 페이지 접속
driver.get("http://www.ecfcsf.or.kr/bbs/board.php?bo_table=notice")

# 공고 목록 가져오기
post_links = driver.find_elements(By.CSS_SELECTOR, "tbody tr")


# 각 게시물에 접속하여 첨부 파일 다운로드
for index, row in enumerate(post_links):
    # 다시 요소를 찾음
    post_links = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
    row = post_links[index]
    link = row.find_element(By.CSS_SELECTOR, "td.link a")

    # 게시물 제목 가져오기
    post_title_element = row.find_element(By.CSS_SELECTOR, "td.link a span.cateTit")
    post_title = post_title_element.text.strip()
    folder_name = os.path.join(base_download_dir, post_title)

    #폴더 생성
    if not os.path.exists(folder_name):
      os.makedirs(folder_name)

    # 폴더에 대한 다운로드 경로 설정
    options.add_experimental_option("prefs", {
        "download.default_directory": folder_name,
        "download.prompt_for_download": False,
        "safebrowsing.enabled": True
    })
    driver = webdriver.Chrome(service=service, options=options)

     # 링크 클릭
    driver.execute_script("arguments[0].click();", link)

    # 페이지가 로드될 때까지 잠시 기다림
    time.sleep(2)

    # 첨부 파일 다운로드 링크 가져오기
    download_links = driver.find_elements(By.CSS_SELECTOR, "tr.bg a.viewFile")

    # 각 첨부 파일 다운로드
    for download_link in download_links:
        file_name = download_link.text
        driver.execute_script("arguments[0].click();", download_link)
        time.sleep(5)  # 파일이 다운로드될 때까지 잠시 기다림

        # 다운로드된 파일 확인 및 이름 변경
        downloaded_files = os.listdir(folder_name)
        downloaded_files.sort(key=lambda x: os.path.getmtime(os.path.join(folder_name, x)), reverse=True)
        newest_file = os.path.join(folder_name, downloaded_files[0])
        new_file_path = os.path.join(folder_name, file_name)

        if os.path.exists(newest_file):
            os.rename(newest_file, new_file_path)
            print(f"{file_name} 다운로드 및 이름 변경 완료!")

    # 뒤로 가기
    driver.back()
    time.sleep(2)  # 페이지가 로드될 때까지 잠시 기다림

# 웹 드라이버 종료
driver.quit()




01_2024년도 제19기 장학생 선발 공고.pdf 다운로드 및 이름 변경 완료!
02_자기소개서.hwp 다운로드 및 이름 변경 완료!
03_체크리스트.hwp 다운로드 및 이름 변경 완료!
04_지원서 작성요령 및 제출서류 준비 안내.pdf 다운로드 및 이름 변경 완료!
1. 제18기 장학생 선발공고문.pdf 다운로드 및 이름 변경 완료!
2. 자기소개서.hwp 다운로드 및 이름 변경 완료!
3. 체크리스트.hwp 다운로드 및 이름 변경 완료!
4. 2023년도 제18기 장학생 지원서 작성요령 및 제출서류 준비 안내.pdf 다운로드 및 이름 변경 완료!
1. 2022년도 제17기 장학생 선발 공고문.pdf 다운로드 및 이름 변경 완료!
2. 지원서 작성요령 및 제출서류 준비.pdf 다운로드 및 이름 변경 완료!
1. 전기공사공제조합장학회 2020년도 제15기 장학생 선발 공고 2019-11-04.hwp 다운로드 및 이름 변경 완료!
2. 지원서 작성요령 및 제출서류 준비.pdf 다운로드 및 이름 변경 완료!
제14기 장학생 선발공고문.hwp 다운로드 및 이름 변경 완료!
06_2018년도 제13기 장학생 선발 공고문.hwp 다운로드 및 이름 변경 완료!
07_장학생 지원서류.hwp 다운로드 및 이름 변경 완료!
09_지원서 작성요령 및 제출서류 준비.pptx 다운로드 및 이름 변경 완료!
양식 다운로드.zip 다운로드 및 이름 변경 완료!
